# Releases b3:

In [1]:
#Bibliotecas Necessárias:
from datetime import datetime,timedelta
import pandas as pd
import numpy as np

In [2]:
#podemos encontrar os xlsx abaixo manualmente pelo link:
# https://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/acoes/consultas/cronograma-de-eventos-corporativos/

#lista de urls que tem os xlsx no site da b3:
url_2021 = "https://www.b3.com.br/data/files/A0/F6/3B/CE/9B3B6710FB5A3B67AC094EA8/Cronograma%20de%20Eventos%20Corporativos%202021.xlsx"
url_2022 = "https://www.b3.com.br/data/files/F5/E5/E7/3A/0EB2E7108BD66BD7AC094EA8/Cronograma%20de%20Eventos%20Corporativos%202022.xlsx"


url = url_2022
raw = pd.read_excel(url) # --> lê o xlsx

In [3]:
raw.head(3)

,NOME DE PREGÃO,SEGMENTO,Demonstrações Financeiras do Exercício,Unnamed: 3,Unnamed: 4,Unnamed: 5,Assembléia Geral Ordinária,Unnamed: 7,Formulário de Referência,Unnamed: 9,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Atualizado,2022-10-27 00:00:00
0,NaN,NaN,Demonstrações Financeiras Anuais Completas,NaN,Padronizadas - DFP,NaN,NaN,NaN,NaN,NaN,...,NaN,1º trimestre,NaN,2º trimestre,NaN,3º trimestre,NaN,NaN,NaN,NaN
1,NaN,NaN,Previsão,Entrega,Previsão,Entrega,Previsão,Realização,Previsão,Entrega,...,Disponibilização do Material,Previsão,Disponibilização do Material,Previsão,Disponibilização do Material,Previsão,Disponibilização do Material,NaN,NaN,NaN
2,3R PETROLEUM,NM,22/02/2022,22/02/2022,22/02/2022,22/02/2022,29/04/2022,29/04/2022,31/05/2022,31/05/2022,...,23/02/2022,04/05/2022,04/05/2022,2022-08-11 00:00:00,2022-08-11 00:00:00,09/11/2022,NaN,NaN,NaN,NaN


In [4]:
raw.iloc[1].index

Index([                                     'NOME DE PREGÃO',
                                                  'SEGMENTO',
                    'Demonstrações Financeiras do Exercício',
                                                'Unnamed: 3',
                                                'Unnamed: 4',
                                                'Unnamed: 5',
                                'Assembléia Geral Ordinária',
                                                'Unnamed: 7',
                                  'Formulário de Referência',
                                                'Unnamed: 9',
                             'Reunião Pública com Analistas',
                                               'Unnamed: 11',
                         'Informações do 1º Trimestre - ITR',
                                               'Unnamed: 13',
                         'Informações do 2º Trimestre - ITR',
                                               'Unnamed: 15',
        

In [5]:
def filtra_trimestres(dt_raw = raw):
    '''
    Descrição: essa função filtra as informações dos releases do restante do dataframe
               e retorna o novo dataframe filtrado.
    Atributo(s): 
             ->dt_raw: recebe por padrão o dataframe extraído do site via url.
    '''

    #Precisamos filtrar as colunas que possuem as strings abaixo:
    quarto_trimestre_ano_passado = 'Demonstrações Financeiras do Exercício'
    primeiro_tri_ano_atual = 'Informações do 1º Trimestre - ITR'
    segundo_tri_ano_atual  = 'Informações do 2º Trimestre - ITR'
    terceiro_tri_ano_atual = 'Informações do 3º Trimestre - ITR'

    #filtrando:
    contador = 0
    for x in dt_raw.iloc[1].index:
        if(str(x) in quarto_trimestre_ano_passado):
            quarto_tri_previsao = contador
            quarto_tri_entrega  = contador + 1

        elif(str(x) in primeiro_tri_ano_atual):
            primeiro_tri_previsao = contador
            primeiro_tri_entrega  = contador + 1

        elif(str(x) in segundo_tri_ano_atual):
            segundo_tri_previsao = contador
            segundo_tri_entrega  = contador + 1
        
        elif(str(x) in terceiro_tri_ano_atual):
            terceiro_tri_previsao = contador
            terceiro_tri_entrega  = contador + 1
        
        contador +=1
    
    #resultado final: coluna inicial (nome das empresas) + colunas das datas dos releases.
    lista = [0 ,quarto_tri_previsao, quarto_tri_entrega,
                 primeiro_tri_previsao, primeiro_tri_entrega,
                 segundo_tri_previsao, segundo_tri_entrega,
                 terceiro_tri_previsao, terceiro_tri_entrega]
    
    return lista

In [6]:
lista_de_trimestes = filtra_trimestres(dt_raw = raw)

In [7]:
lista_de_trimestes

[0, 2, 3, 12, 13, 14, 15, 16, 17]

In [8]:
def organiza_dataframe(lista_de_trimestes = lista_de_trimestes, dt_filtrado=raw):
    '''
    Descrição: organiza o dataframe de forma a eliminar informações desnecessárias.
    Atributo(s):
             ->lista_de_trimestes: recebe a lista adquirida na função filtra_trimestres.
                                   essa lista nos diz a posição das colunas que precisamos filtrar.
             ->dt_filtrado: recebe o dataframe inicial extraido da url do site da b3.
    '''

    #filtra as colunas:
    dt_filtrado = raw.iloc[2:,lista_de_trimestes]
    #modifica o nome das colunas:
    dt_filtrado.columns = ['EMPRESA','4Q_ANTIGO_PREVISAO', '4Q_ANTIGO_ENTREGA', '1Q_ATUAL_PREVISAO', '1Q_ATUAL_ENTREGA',
                                '2Q_ATUAL_PREVISAO', '2Q_ATUAL_ENTREGA', '3Q_ATUAL_PREVISAO', '3Q_ATUAL_ENTREGA']
    
    #faz a coluna com o nome das empresas virar índice e remove ela como coluna:
    dt_filtrado.index = dt_filtrado.EMPRESA
    del dt_filtrado['EMPRESA']
    
    return dt_filtrado

In [9]:
dataframe_organizado = organiza_dataframe()
dataframe_organizado

,4Q_ANTIGO_PREVISAO,4Q_ANTIGO_ENTREGA,1Q_ATUAL_PREVISAO,1Q_ATUAL_ENTREGA,2Q_ATUAL_PREVISAO,2Q_ATUAL_ENTREGA,3Q_ATUAL_PREVISAO,3Q_ATUAL_ENTREGA
EMPRESA,,,,,,,,
3R PETROLEUM,22/02/2022,22/02/2022,03/05/2022,03/05/2022,2022-08-10 00:00:00,2022-08-10 00:00:00,08/11/2022,NaN
3TENTOS,17/02/2022,2022-02-17 00:00:00,2022-05-05 00:00:00,2022-05-05 00:00:00,2022-08-11 00:00:00,2022-08-11 00:00:00,07/11/2022,NaN
ABC BRASIL,2022-02-11 00:00:00,2022-02-11 00:00:00,2022-05-09 00:00:00,2022-05-09 00:00:00,2022-08-09 00:00:00,2022-08-09 00:00:00,2022-11-04 00:00:00,NaN
AERIS,17/02/2022,2022-02-17 00:00:00,12/05/2022,12/05/2022,11/08/2022,11/08/2022,10/11/2022,NaN
AES BRASIL,2022-03-03 00:00:00,2022-03-03 00:00:00,05/05/2022,05/05/2022,04/08/2022,04/08/2022,03/11/2022,NaN
...,...,...,...,...,...,...,...,...
WESTWING,2022-03-22 00:00:00,2022-03-22 00:00:00,2022-05-11 00:00:00,2022-05-11 00:00:00,2022-08-15 00:00:00,2022-08-15 00:00:00,10/11/2022,NaN
WILSON SONS,23/03/2022,23/03/2022,13/05/2022,13/05/2022,10/08/2022,10/08/2022,09/11/2022,NaN
WIZ S.A.,09/03/2022,2022-03-10 00:00:00,12/05/2022,12/05/2022,11/08/2022,11/08/2022,10/11/2022,NaN


In [10]:
#separando o dataframe anterior em 2 dataframes, um para as datas previstas e outro para as datas entregues:
dt_previsao = dataframe_organizado[['4Q_ANTIGO_PREVISAO', '1Q_ATUAL_PREVISAO', '2Q_ATUAL_PREVISAO', '3Q_ATUAL_PREVISAO']]
dt_entrega = dataframe_organizado[['4Q_ANTIGO_ENTREGA', '1Q_ATUAL_ENTREGA', '2Q_ATUAL_ENTREGA', '3Q_ATUAL_ENTREGA']]

dt_previsao

,4Q_ANTIGO_PREVISAO,1Q_ATUAL_PREVISAO,2Q_ATUAL_PREVISAO,3Q_ATUAL_PREVISAO
EMPRESA,,,,
3R PETROLEUM,22/02/2022,03/05/2022,2022-08-10 00:00:00,08/11/2022
3TENTOS,17/02/2022,2022-05-05 00:00:00,2022-08-11 00:00:00,07/11/2022
ABC BRASIL,2022-02-11 00:00:00,2022-05-09 00:00:00,2022-08-09 00:00:00,2022-11-04 00:00:00
AERIS,17/02/2022,12/05/2022,11/08/2022,10/11/2022
AES BRASIL,2022-03-03 00:00:00,05/05/2022,04/08/2022,03/11/2022
...,...,...,...,...
WESTWING,2022-03-22 00:00:00,2022-05-11 00:00:00,2022-08-15 00:00:00,10/11/2022
WILSON SONS,23/03/2022,13/05/2022,10/08/2022,09/11/2022
WIZ S.A.,09/03/2022,12/05/2022,11/08/2022,10/11/2022


In [11]:
#empresa , coluna
dt_previsao.loc['3TENTOS', '4Q_ANTIGO_PREVISAO']

'17/02/2022'

In [12]:
# agora vamos criar um dataframe onde vamos comparar ambas as datas da seguinte maneira:
# se tiver dados de entrega, mostrar, senão mostrar os dados previstos
dt_data_mais_recente = dt_entrega.copy()

for coluna in range(len(dt_data_mais_recente.columns)):
    for empresa in range(len(dt_data_mais_recente.index)):
        # de acordo com o python nan != nan da False, logo verificamos se o campo está como nan, se sim substituimos o valor, do contrário
        # iremos ficar com o dado da entrega:
        if(dt_data_mais_recente.iloc[empresa, coluna] != dt_data_mais_recente.iloc[empresa, coluna]):
            dt_data_mais_recente.iloc[empresa, coluna] = dt_previsao.iloc[empresa, coluna]




In [13]:
##dt_data_mais_recente2 = pd.DataFrame()
#for coluna in range(len(dt_data_mais_recente.columns)):
#    for empresa in range(len(dt_data_mais_recente.index)):
#        #converte todos os campos para datetime e caso o campo seja invalido vamos apagar ele da lista 
#        #as vezes podem vir datas invalidas como novembro com 31 dias
#        try:
#            dt_data_mais_recente.iloc[empresa, coluna] = pd.to_datetime(dt_data_mais_recente.iloc[empresa, coluna], format="%d/%m/%Y")
#        except:
#            pass
#            #print("erro em " + str(coluna) + " coluna, " + str(empresa) + " empresa, " + str(dt_data_mais_recente))


In [14]:
#por alguma razão a b3 colocou a mesma empresa duplicada, então vamos remover ela manualmente:
#dt_data_mais_recente.drop('AMBIPAR', inplace=True)

In [15]:
dt_data_mais_recente.head(5)

,4Q_ANTIGO_ENTREGA,1Q_ATUAL_ENTREGA,2Q_ATUAL_ENTREGA,3Q_ATUAL_ENTREGA
EMPRESA,,,,
3R PETROLEUM,22/02/2022,03/05/2022,2022-08-10 00:00:00,08/11/2022
3TENTOS,2022-02-17 00:00:00,2022-05-05 00:00:00,2022-08-11 00:00:00,07/11/2022
ABC BRASIL,2022-02-11 00:00:00,2022-05-09 00:00:00,2022-08-09 00:00:00,2022-11-04 00:00:00
AERIS,2022-02-17 00:00:00,12/05/2022,11/08/2022,10/11/2022
AES BRASIL,2022-03-03 00:00:00,05/05/2022,04/08/2022,03/11/2022


In [16]:
lista_nome_release = list()
lista_data_release = list()

for coluna in dt_data_mais_recente.columns:
    for empresa in dt_data_mais_recente.index:
        lista_nome_release.append(str(empresa) + '-' + str(coluna))
        lista_data_release.append(dt_data_mais_recente.loc[empresa, coluna])
        #print(str(empresa) + '-' + str(coluna))
        #print(dt_data_mais_recente.loc[empresa, coluna])

In [17]:
teste = pd.DataFrame(index=lista_nome_release, data=lista_data_release)
teste.columns = ['DT']
teste.index.name = 'EMPRESA'

In [18]:
teste

,DT
EMPRESA,
3R PETROLEUM-4Q_ANTIGO_ENTREGA,22/02/2022
3TENTOS-4Q_ANTIGO_ENTREGA,2022-02-17 00:00:00
ABC BRASIL-4Q_ANTIGO_ENTREGA,2022-02-11 00:00:00
AERIS-4Q_ANTIGO_ENTREGA,2022-02-17 00:00:00
AES BRASIL-4Q_ANTIGO_ENTREGA,2022-03-03 00:00:00
...,...
WESTWING-3Q_ATUAL_ENTREGA,10/11/2022
WILSON SONS-3Q_ATUAL_ENTREGA,09/11/2022
WIZ S.A.-3Q_ATUAL_ENTREGA,10/11/2022


In [19]:
#este bloco transforma todos os valores para datetime e caso haja algum valor invalido como 31/11/2022
#ignora esse valor e nao o utiliza no dataframe final.
list_empresas = list()
list_datas = list()

for x in range(len(teste.DT)):
    try:
        list_datas.append(pd.to_datetime(teste.iloc[x].DT, format="%d/%m/%Y"))
        list_empresas.append(teste.iloc[x].name)
    except:
        pass
output = pd.DataFrame(list(zip(list_empresas,list_datas)), columns=["EMPRESA", "DT"])

In [20]:
output.sort_values(by='DT', ascending=True)
output

,EMPRESA,DT
0,3R PETROLEUM-4Q_ANTIGO_ENTREGA,2022-02-22
1,3TENTOS-4Q_ANTIGO_ENTREGA,2022-02-17
2,ABC BRASIL-4Q_ANTIGO_ENTREGA,2022-02-11
3,AERIS-4Q_ANTIGO_ENTREGA,2022-02-17
4,AES BRASIL-4Q_ANTIGO_ENTREGA,2022-03-03
...,...,...
1145,WESTWING-3Q_ATUAL_ENTREGA,2022-11-10
1146,WILSON SONS-3Q_ATUAL_ENTREGA,2022-11-09
1147,WIZ S.A.-3Q_ATUAL_ENTREGA,2022-11-10
1148,YDUQS PART-3Q_ATUAL_ENTREGA,2022-11-10


In [22]:
#passando pro excel:
dt_data_mais_recente.to_excel('releases_b3.xlsx')